## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-07-13-07-58 +0000,nyt,As Trump Administration Plans to Burn Contrace...,https://www.nytimes.com/2025/08/07/world/europ...
1,2025-08-07-13-03-08 +0000,bbc,"Trump and Putin to meet in coming days, Kremli...",https://www.bbc.com/news/articles/cr5rdl1y8ndo...
2,2025-08-07-13-03-00 +0000,wsj,Stock Market Today: Dow Futures Rise After New...,https://www.wsj.com/livecoverage/stock-market-...
3,2025-08-07-13-00-02 +0000,nyt,"India Tariffs Could Make Spices Costlier, Trad...",https://www.nytimes.com/2025/08/07/business/in...
4,2025-08-07-12-56-14 +0000,nyt,Wind and Solar Projects Stall as Trump Cracks ...,https://www.nytimes.com/2025/08/07/climate/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2460/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,74
19,tariffs,21
18,new,18
7,putin,17
23,india,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
295,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...,159
230,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...,138
303,2025-08-06-14-50-35 +0000,nypost,Apple commits another $100B to US manufacturin...,https://nypost.com/2025/08/06/business/apple-c...,134
195,2025-08-06-21-56-54 +0000,wapo,"Trump raises tariffs on India to 50 percent, c...",https://www.washingtonpost.com/business/2025/0...,132
60,2025-08-07-10-05-00 +0000,wsj,Russia Says Putin and Trump Will Meet in Comin...,https://www.wsj.com/world/russia/russia-confir...,131


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
295,159,2025-08-06-15-28-28 +0000,bbc,Trump-Modi ties hit rock bottom with new tarif...,https://www.bbc.com/news/articles/c15lqe7v302o...
230,64,2025-08-06-19-46-31 +0000,nypost,"Trump plans sitdown with Putin, Zelensky in fi...",https://nypost.com/2025/08/06/us-news/trump-pl...
129,55,2025-08-07-04-12-00 +0000,wsj,"China’s exports grew at a faster clip in July,...",https://www.wsj.com/economy/trade/chinas-expor...
151,43,2025-08-07-01-00-00 +0000,wsj,Economic activity tied to manufacturing has sh...,https://www.wsj.com/economy/trump-pledged-to-b...
42,40,2025-08-07-11-06-24 +0000,bbc,"First migrants detained under 'one in, one out...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
60,40,2025-08-07-10-05-00 +0000,wsj,Russia Says Putin and Trump Will Meet in Comin...,https://www.wsj.com/world/russia/russia-confir...
269,39,2025-08-06-16-56-15 +0000,nypost,Italy approves $15.5 billion project to build ...,https://nypost.com/2025/08/06/world-news/italy...
110,36,2025-08-07-06-51-12 +0000,nypost,Trump celebrates ‘billions of dollars’ floodin...,https://nypost.com/2025/08/07/us-news/trumps-h...
231,35,2025-08-06-19-42-41 +0000,latimes,Sea lion bites young surfer in Monterey County...,https://www.latimes.com/california/story/2025-...
181,35,2025-08-06-22-30-27 +0000,startribune,Tim Walz wants Democrats to fight Texas redist...,https://www.startribune.com/tim-walz-wants-dem...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
